In [1]:
import pandas as pd
import numpy as np
import requests
import csv

In [2]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text #requests.get(url).text will ping a website and return you HTML of the website.

We begin by reading the source code for a given web page and creating a BeautifulSoup (soup)object with the BeautifulSoup function. Beautiful Soup is a Python package for parsing HTML and XML documents. It creates a parse tree for parsed pages that can be used to extract data from HTML, which is useful for web scraping. Prettify() function in BeautifulSoup will enable us to view how the tags are nested in the document.

In [4]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url,'html.parser')
#print(soup.prettify())

our first task is to find class "wikitable sortable" in the HTML script.

In [5]:
My_table = soup.find('table',{'class':'wikitable sortable'})# We can get this by inspecting the Wikipedia page

Now to extract all the links within 'tr tag', we will use find_all().

In [6]:
rows = My_table.find_all('tr')

In [8]:
header = [th.text.rstrip() for th in rows[0].find_all('th')]

with open('output.csv', 'w') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(header)
    for row in rows[1:]:
        data = [td.text.rstrip() for td in row.find_all('td')]
        writer.writerow(data)


The first row ussually contains the header cells. We serch throught the first row in the rows list to get the text values of all th elements in that row.
we also ensure to remove the all trailing whitespaces in the text using the <b>rstrip</b> python string method.
The w mode is used to ensure the file is open for writing.
First we write the header row, then loop through the rest of the rows ignoring the first row to get the data contained within and write the data for all those rows to the file object.

In [9]:
df = pd.read_csv('output.csv')
df = df[df['Borough']!='Not assigned']#process only those rows where 'Borough' is not 'Not assigned'
df.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [10]:
df1 = pd.DataFrame(df.groupby('Postal Code')['Postal Code'].count()>1)
df1.reset_index(drop=True, inplace=True)#Dropping the index column of a pandas.DataFrame removes the index column from the DataFrame and replaces it with the standard sequential indexing.
df1[df1['Postal Code']==True]

,Postal Code


From above code we found out that there are no repeating postal codes.

In [11]:
df.shape

(103, 3)

Now we are going to fetch the geospatial data for the given postal codes.

In [1]:
!wget -O geodata.csv http://cocl.us/Geospatial_data

--2020-05-08 13:00:50--  http://cocl.us/Geospatial_data
Resolving cocl.us (cocl.us)... 159.8.69.24, 159.8.72.228, 159.8.69.21
Connecting to cocl.us (cocl.us)|159.8.69.24|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cocl.us/Geospatial_data [following]
--2020-05-08 13:00:50--  https://cocl.us/Geospatial_data
Connecting to cocl.us (cocl.us)|159.8.69.24|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-05-08 13:00:53--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 185.235.236.197
Connecting to ibm.box.com (ibm.box.com)|185.235.236.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-05-08 13:00:53--  https://ibm.box.com/public/static/

In [12]:
geodata = pd.read_csv('geodata.csv')
geodata.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [16]:
df_final = pd.merge(df, geodata, on = 'Postal Code', how = 'inner')#merge the postal data with the geospatial data using inner join on 'Postal Code' column
df_final

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
